In [38]:
import pandas as pd
import numpy as np
import random
import pickle

from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [39]:
df1 = pd.read_csv('Fake.csv')
df2 = pd.read_csv('True.csv')

In [40]:
df = pd.concat([df1,df2])

In [41]:
df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [42]:
df.shape

(44898, 4)

In [43]:
text = list(df.text.values)
full_text = " ".join(text)

In [44]:
len(full_text)

110902984

In [60]:
full_text = full_text[:1000000] #18679895

In [61]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(full_text.lower())

In [62]:
unique_tokens = np.unique(tokens)
unique_tokens_index = {token:indx for indx,token in enumerate(unique_tokens)}

In [63]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens)-n_words):
    input_words.append(tokens[i:i+n_words])
    next_words.append(tokens[i+n_words])

In [64]:
x = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [65]:
y.shape

(17221, 3596)

In [85]:
for i,words in enumerate(input_words):
    for j,word in enumerate(words):
        x[i,j,unique_tokens_index[word]]=1
    y[i, unique_tokens_index[next_words[i]]]=1

In [86]:
model = Sequential()
model.add(LSTM(256, input_shape = (n_words,len(unique_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [90]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])

In [91]:
model.fit(x, y, batch_size=128, epochs=50, shuffle=True)

Epoch 1/50
270/270 [==============================] - 34s 115ms/step - loss: 7.1449 - accuracy: 0.0452
Epoch 2/50
270/270 [==============================] - 31s 116ms/step - loss: 6.7560 - accuracy: 0.0457
Epoch 3/50
270/270 [==============================] - 32s 119ms/step - loss: 6.7264 - accuracy: 0.0457
Epoch 4/50
270/270 [==============================] - 36s 133ms/step - loss: 6.6577 - accuracy: 0.0457
Epoch 5/50
270/270 [==============================] - 35s 129ms/step - loss: 6.6502 - accuracy: 0.0452
Epoch 6/50
270/270 [==============================] - 34s 127ms/step - loss: 6.5117 - accuracy: 0.0479
Epoch 7/50
270/270 [==============================] - 34s 125ms/step - loss: 6.3769 - accuracy: 0.0522
Epoch 8/50
270/270 [==============================] - 34s 125ms/step - loss: 6.2173 - accuracy: 0.0566
Epoch 9/50
270/270 [==============================] - 34s 125ms/step - loss: 6.0399 - accuracy: 0.0620
Epoch 10/50
270/270 [==============================] - 34s 124ms/step - l

In [92]:
def predict(input_text,n_best):
    input_text = input_text.lower()
    x = np.zeros((1,20,len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        x[0, i, unique_tokens_index[word]]=1
    prediction = model.predict(x)[0]
    return np.argpartition(prediction, -n_best)[-n_best:]

In [93]:
def generate_text(input_text,text_lenght, creativity=3):
    word_seq = input_text.split()
    current = 0
    for i in range(text_lenght):
        sub_seq = " ".join(tokenizer.tokenize(" ".join(word_seq).lower()[current:current+n_words]))
        try:
            choice = unique_tokens[random.choice(predict(sub_seq, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_seq.append(choice)
        current = current+1
    return " ".join(word_seq)

In [95]:
generate_text("I want to see what is possible in this world and want to visit countries like India and dubai and",5,1)

'I want to see what is possible in this world and want to visit countries like India and dubai and placed durbin school thanking collection'